In [1]:
import pandas as pd

# Load the dataset
file_path = '../data/products_books_v1_10_10.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())


from gensim.models import Word2Vec
import gensim
import string


           id                                      product_title  \
0  0002005018                                       Clara Callan   
1  0399135782                             The Kitchen God's Wife   
2  0440234743                                      The Testament   
3  0452264464               Beloved (Plume Contemporary Fiction)   
4  0609804618  Our Dumb Century: The Onion Presents 100 Years...   

                                       product_image  product_price  \
0  http://images.amazon.com/images/P/0002005018.0...            NaN   
1  http://images.amazon.com/images/P/0399135782.0...            NaN   
2  http://images.amazon.com/images/P/0440234743.0...            NaN   
3  http://images.amazon.com/images/P/0452264464.0...            NaN   
4  http://images.amazon.com/images/P/0609804618.0...            NaN   

                                        product_soup  product_tags  
0  Clara Callan Richard Bruce Wright HarperFlamin...           NaN  
1    The Kitchen God's Wif

In [6]:

# Preprocess text data
data['processed_soup'] = data['product_soup'].str.lower().str.translate(str.maketrans('', '', string.punctuation))

# Prepare data for Word2Vec model
sentences = [row.split() for row in data['processed_soup'].dropna()]

# Train Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for later use
model.save("../data/book_wordvec.model")



In [10]:
# Updated function to recommend books based on similar word vectors
def recommend_books_updated(input_text, top_n=5):
    input_text = input_text.lower().translate(str.maketrans('', '', string.punctuation)).split()
    vector = model.wv[input_text].mean(axis=0)
    similar_vectors = model.wv.similar_by_vector(vector, topn=top_n + 10)  # Retrieve more results to filter unique titles
    recommended_titles = []
    for book_vector in similar_vectors:
        similar_title = data.loc[data['processed_soup'].apply(lambda x: any(word in x for word in input_text)), 'product_title'].unique()
        for title in similar_title:
            if title not in recommended_titles and len(recommended_titles) < top_n:
                recommended_titles.append(title)
    return recommended_titles


# Test the updated function with the same book title
recommend_books_updated("Clara Callan")

['Clara Callan',
 'Cunt: A Declaration of Independence (Live Girls)',
 'Callander Square',
 'Cunt: A Declaration of Independence (Live Girls Series)']

In [14]:

# Test the updated function Kings the same book title
recommend_books_updated("Kings")

['Pigs in Heaven',
 'Poisonwood Bible Edition Uk',
 'The Game of Kings (Lymond Chronicles, 1)',
 'The Bean Trees',
 'Homeland and Other Stories']